In [1]:
import pandas as pd

file_path = 'preprocessed_data.csv'
data = pd.read_csv(file_path)

data.head()

,processed_text,emotion
0,thought there top filmed elimination make say ...,neutral
1,yes going lieda,approval
2,huh,curiosity
3,follow account tumblr satire basically incel c...,amusement
4,thought ah meant ad general sorry bud,remorse


In [2]:
cleaned_data = data.dropna(subset=["processed_text", "emotion"])
cleaned_data = cleaned_data[cleaned_data["processed_text"].str.strip() != ""]

original_count = len(data)
cleaned_count = len(cleaned_data)

cleaned_data.to_csv('cleaned_data.csv', index=False)

original_count, cleaned_count

(319580, 209680)

In [5]:
!pip install dataset

In [6]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

data_path = 'cleaned_data.csv'
data = pd.read_csv(data_path)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    data["processed_text"].tolist(),
    data["emotion"].tolist(),
    test_size=0.2,
    random_state=42
)

unique_labels = sorted(set(train_labels + val_labels))
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

train_labels = [label_to_id[label] for label in train_labels]
val_labels = [label_to_id[label] for label in val_labels]

model_name = "nateraw/bert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels}).map(tokenize_function, batched=True)
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels}).map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels),
    id2label=id_to_label,
    label2id=label_to_id,
    ignore_mismatched_sizes=True
)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model("./emotion_detection_model")
tokenizer.save_pretrained("./emotion_detection_model")

print("Model fine-tuning complete and saved to './emotion_detection_model'")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.873900,1.852283
2,1.838900,1.760825
3,1.549700,1.763313


Model fine-tuning complete and saved to './emotion_detection_model'


In [8]:
!zip -r emotion_detection_model.zip emotion_detection_model

  adding: emotion_detection_model/ (stored 0%)
  adding: emotion_detection_model/training_args.bin (deflated 51%)
  adding: emotion_detection_model/tokenizer_config.json (deflated 75%)
  adding: emotion_detection_model/vocab.txt (deflated 53%)
  adding: emotion_detection_model/model.safetensors (deflated 7%)
  adding: emotion_detection_model/special_tokens_map.json (deflated 42%)
  adding: emotion_detection_model/tokenizer.json (deflated 71%)
  adding: emotion_detection_model/config.json (deflated 60%)


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = "./emotion_detection_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

model.eval()

id_to_label = model.config.id2label

def predict_emotion(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()

    return id_to_label[predicted_class_id]

sample_texts = [
    "I can't believe I got the promotion! This is a dream come true after all the hard work.",
    "The loss of my pet has left an emptiness I can't explain. It's been a tough few weeks.",
    "I'm beyond furious with the way customer service handled my issue—it was utterly unacceptable!",
    "Honestly, I feel so uncertain about the future, like I’m just drifting without purpose.",
    "The sunset was so beautiful today—it reminded me how grateful I am for small moments of peace.",
    "It's strange. I feel both excited and scared about starting a new chapter in my life.",
    "I’ve been feeling neglected lately, like no one really notices or cares about me anymore.",
    "What an amazing surprise! I didn’t expect my friends to throw such a thoughtful party.",
    "It's been a monotonous day, nothing exciting, just going through the motions as usual.",
    "I can’t describe how anxious I feel about the upcoming interview—it’s overwhelming."
]

for text in sample_texts:
    print(f"Text: {text}")
    print(f"Predicted Emotion: {predict_emotion(text)}")
    print()


Text: I can't believe I got the promotion! This is a dream come true after all the hard work.
Predicted Emotion: optimism

Text: The loss of my pet has left an emptiness I can't explain. It's been a tough few weeks.
Predicted Emotion: sadness

Text: I'm beyond furious with the way customer service handled my issue—it was utterly unacceptable!
Predicted Emotion: anger

Text: Honestly, I feel so uncertain about the future, like I’m just drifting without purpose.
Predicted Emotion: nervousness

Text: The sunset was so beautiful today—it reminded me how grateful I am for small moments of peace.
Predicted Emotion: gratitude

Text: It's strange. I feel both excited and scared about starting a new chapter in my life.
Predicted Emotion: excitement

Text: I’ve been feeling neglected lately, like no one really notices or cares about me anymore.
Predicted Emotion: disappointment

Text: What an amazing surprise! I didn’t expect my friends to throw such a thoughtful party.
Predicted Emotion: admira